In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import numpy as np
import cv2
from PIL import Image

import cc3d

In [ ]:
#値を-1から1に正規化する関数
def normalize_x(image):
    return image / 127.5 - 1


def denormalize_x(image):
    return (image + 1) * 127.5


#値を0から1正規化する関数
def normalize_y(image):
    return image / 255


#値を0から255に戻す関数
def denormalize_y(image):
    return image * 255

In [ ]:
def load_Y_gray_with_gaussian(folder_path, thresh = None , normalize = True, g_size = None):
    image_files = []    
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else:
            pass
        
    image_files.sort()
    print(image_files)
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)
    
    images = np.zeros(
        (len(image_files), img.shape[0], img.shape[1], 1) ,np.float32
    )
    
    for i , image_file in enumerate(image_files):
        image = cv2.imread(
            folder_path + os.sep + image_file ,
            cv2.IMREAD_GRAYSCALE
        )

        if g_size:
            image = cv2.GaussianBlur(
                image, (g_size, g_size), 0
            )
                
        if thresh:
            ret , image = cv2.threshold(image , thresh , 255 , cv2.THRESH_BINARY)
        image = image[ : , : , np.newaxis]
        if normalize:
            images[i] = normalize_y(image)
        else:
            images[i] = image
            
    print(images.shape)
    
    return images , image_files

def load_X_gray(folder_path):
    
    image_files = []       
    for file in os.listdir(folder_path):
        base, ext = os.path.splitext(file)
        if ext == '.png':
            image_files.append(file)
        else :
            pass
        
    image_files.sort()
    print (image_files)
    
    img = cv2.imread(folder_path + os.sep + image_files[0], cv2.IMREAD_GRAYSCALE)

    images = np.zeros((len(image_files), img.shape[0], img.shape[1], 1), np.float32)
    for i, image_file in enumerate(image_files):
        image = cv2.imread(folder_path + os.sep + image_file, cv2.IMREAD_GRAYSCALE)
        image = image[:, :, np.newaxis]
        images[i] = normalize_x(image)
    return images, image_files


In [ ]:
def make_mask_img(ori_img, mask_img):
    mask_img_rgb = np.zeros((mask_img.shape[0], mask_img.shape[1], 3), np.float32)
    mask_img_rgb[:,:,2] = mask_img[:,:,0]
    mask_img_rgb[:,:,0] = mask_img[:,:,0]
    masked_img = cv2.addWeighted(mask_img_rgb,0.5,cv2.cvtColor(ori_img+0.75, cv2.COLOR_GRAY2BGR),0.6,0)
    return masked_img

In [ ]:
ori_imgs, _ = load_X_gray("H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/dataset/cropped_003/resize_10x10x10/")
merge_003_imgs, _ = load_Y_gray_with_gaussian_and_numeric("H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_003/merged_003//")
labeled_003_imgs, _ =load_Y_gray_with_gaussian_and_numeric("H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_003/modified_003/")
ori_80_imgs, _ = load_X_gray("H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_003/original//")
merge_004_imgs, _ = load_Y_gray_with_gaussian_and_numeric("H:/DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/cropped_003/merged_004///")


In [ ]:
out_dir = "Z:DeepLearningData/research_010_NIH3T3/shCtrl_001_realignment/active_learning_dataset/for_paper/cropped_003_mito_mito_fig1_d"

os.makedirs(out_dir, exist_ok = True)

for i in range(6):
    grey_ori_img = denormalize_x(ori_80_imgs[29+i][570:650, 570:650])
    rgb_ori_img = np.zeros((grey_ori_img.shape[0], grey_ori_img.shape[1], 3), np.float32)
    rgb_ori_img[:,:,0] = grey_ori_img[:,:,0]
    rgb_ori_img[:,:,1] = grey_ori_img[:,:,0]
    rgb_ori_img[:,:,2] = grey_ori_img[:,:,0]
    
    cv2.imwrite(
        f"{out_dir}/{str(29+i).zfill(4)}_ori.png",
        rgb_ori_img
    )
    
    cv2.imwrite(
        f"{out_dir}/{str(29+i).zfill(4)}_pred_003.png",
        denormalize_y(merge_003_imgs[29+i][570:650, 570:650] *255)
    )
    
    cv2.imwrite(
        f"{out_dir}/{str(29+i).zfill(4)}_pred_004.png",
        denormalize_y(merge_004_imgs[59+i][680:760, 723:803] * 255)
    )
    
    cv2.imwrite(
        f"{out_dir}/{str(29+i).zfill(4)}_masked_003.png",
        denormalize_y(make_mask_img(ori_80_imgs[29+i][570:650, 570:650], merge_003_imgs[29+i][570:650, 570:650]*  255) )
    )    

    cv2.imwrite(
        f"{out_dir}/{str(29+i).zfill(4)}_masked_004.png",
        denormalize_y(make_mask_img(ori_80_imgs[29+i][570:650, 570:650], merge_004_imgs[59+i][680:760, 723:803] * 255))
    )  
    

In [ ]:
cv2.imwrite(
    f"{out_dir}/{str(32).zfill(4)}_label_003.png",
    denormalize_y(make_mask_img(ori_80_imgs[32][570:650, 570:650], labeled_003_imgs[32][570:650, 570:650]*  255) )
)